# Analyse de LMGEC

> Article analysé : **LMGEC: Simultaneous Linear Multi-view Attributed Graph Representation Learning and Clustering**, WSDM 2023.


---


## Problématique centrale  :  
LMGEC répond à la problématique suivante :
“Comment apprendre des représentations partagées et effectuer un clustering efficace sur des graphes multi-vues attribués, tout en garantissant simplicité, rapidité, et robustesse face à l’hétérogénéité des vues ?”
>

![Alt text](problem.png)


## 📅 1. Méthodologie : quel type de fusion ?

LMGEC repose sur une **fusion linéaire pondée tardive** des vues. 

- Chaque vue est d'abord **filtrée localement** (1-hop) pour lisser les attributs : $$ H_v = S_v X_v $$
- Une **pondération adaptative** des vues est appliquée via des poids $$ \alpha_v $$, calculés dynamiquement.
- Les embeddings obtenus sont projetés et **fusionnés dans un espace commun** pour effectuer le clustering.

> **Type de fusion :** tardive + pondération adaptative (soft fusion)

---

## 🔍 2. Hypothèses sur les vues

- ✅ Mêmes nœuds dans toutes les vues
- ✅ Vues potentiellement **très hétérogènes** (topologies ou attributs)
- ❌ Pas de traitement spécial des vues **manquantes** ou désalignées
- ✅ Le modèle peut atténuer les vues bruitées via la pondération $$ \alpha_v $$

---

## 📊 3. Modèles mathématiques

- **Filtrage de chaque vue :** $$ H_v = S_v X_v $$, où $$ S_v = \tilde{D}^{-1} (\tilde{A}_v) $$ avec self-loops.

- **Objectif :**
$$
\min_{G, F, W_1,\dots,W_V} \sum_{v=1}^{V} \alpha_v \| H_v - G F W_v^\top \|^2
$$
Avec :
  - $ G \in \{0,1\}^{n \times k} $ : clustering (soft or hard)
  - $ W_v \in \mathbb{R}^{d \times f} $, $ W_v W_v^\top = I $

- **Pondération des vues :**
$$
\alpha_v = \text{softmax}\left(-\frac{I_v}{\tau}\right), \quad I_v = \| H_v - G_v F_v \|
$$

> Optimisation par **Bloc Coordinate Descent**

> 
![Alt text](recon.png)
---

## 📈 4. Types de données utilisées

- **Topologies différentes, mêmes features :** ACM, DBLP, IMDB
- **Mêmes topologies, features différentes :** Amazon Photos
- **Topologies + features différentes :** Wiki

> LMGEC couvre **tous les cas multi-vues usuels**
> 
![Alt text](topo.png)
---

## 📊 5. Métriques d’évaluation

Les performances sont mesurées avec 4 métriques standards en clustering :

| Métrique | Description |
|----------|-------------|
| **NMI** | Normalized Mutual Information |
| **ARI** | Adjusted Rand Index |
| **ACC** | Accuracy (appliquée au clustering) |
| **F1-score** | Précision + rappel |

---


# 📊 Métriques d’évaluation pour le clustering non supervisé

Lorsque j’évalue un algorithme de clustering (comme LMGEC), je dois utiliser des métriques quantitatives pour mesurer à quel point les clusters trouvés sont cohérents avec les classes réelles (si elles sont disponibles). Voici les principales métriques utilisées :

---

| **Métrique** | **Nom complet** | **Ce qu’elle mesure** | **Valeurs typiques** |
|--------------|------------------|------------------------|-----------------------|
| **NMI** | Normalized Mutual Information | Le degré de similarité entre les clusters prédits et les vraies classes, basé sur l’information partagée | Entre 0 (aucune info) et 1 (parfait) |
| **ARI** | Adjusted Rand Index | À quel point deux partitions (clusters vs classes) sont en accord, corrigé pour le hasard | Entre -1 (pire que hasard) et 1 (parfait accord) |
| **ACC** | Clustering Accuracy | Pourcentage d’exemples correctement classés, après réassignation optimale des labels | Entre 0 et 1 (ou 0% à 100%) |
| **F1-score** | F1 Measure (harmonique précision / rappel) | Moyenne harmonique entre la précision (exactitude des clusters) et le rappel (couverture des vraies classes) | Entre 0 et 1 |

---

## 🧠 Détails complémentaires

### 🔹 NMI (Normalized Mutual Information)
- Formule :
$$
\text{NMI}(Y, C) = \frac{2 \, I(Y; C)}{H(Y) + H(C)}
$$
avec \( I(Y; C) \) l'information mutuelle entre la partition vraie \( Y \) et prédite \( C \).
- Avantage : **invariante aux permutations de labels**.

### 🔹 ARI (Adjusted Rand Index)
- Tient compte du **nombre de paires** correctement regroupées ou séparées.
- Corrige le **Rand Index** pour compenser le regroupement aléatoire.
- Avantage : bonne robustesse même avec des déséquilibres de classe.

### 🔹 ACC (Accuracy)
- Mesure directe et intuitive.
- Requiert une **permutation optimale des clusters** (car les labels peuvent être inversés).

### 🔹 F1-score
- Combine deux notions :
  - **Précision** : parmi les éléments du cluster, combien sont bien classés ?
  - **Rappel** : parmi les vrais éléments d’une classe, combien sont capturés ?
- L’équilibre entre les deux est utile surtout pour les classes déséquilibrées.

---

> En résumé : utiliser plusieurs métriques permet d’avoir une vue plus complète de la qualité du clustering. NMI et ARI sont les plus stables pour comparer les méthodes, tandis que ACC et F1 donnent une interprétation plus intuitive des performances.





# 🎯 Contribution originale et évaluation critique de LMGEC

## 🧩 Quelle est la **contribution originale** du papier ?

L’article propose **LMGEC**, un modèle linéaire simple et efficace pour :

- Réaliser **simultanément** l’apprentissage de représentation et le clustering sur des graphes multi-vues attribués.
- Offrir une **formulation unifiée** intégrant :
  - une étape de **propagation locale (1-hop)** par un filtre de graphe linéaire,
  - un **mécanisme de pondération adaptative des vues** (softmax sur l’inertie),
  - un objectif combiné de **reconstruction + clustering**.
- Être **générique**, applicable à :
  - plusieurs graphes avec une même matrice de features,
  - plusieurs matrices de features sur un seul graphe,
  - ou un mélange des deux (cas du dataset Wiki).
- Être **beaucoup plus rapide** que les modèles existants tout en offrant des performances comparables, voire meilleures.
- Fournir une **analyse mathématique et expérimentale approfondie** ainsi que le **code open-source**.

## ✅ **Points forts** de LMGEC

| Atout | Détail |
|-------|--------|
| ✅ Simplicité | Formulation linéaire claire et interprétable |
| ✅ Efficacité | Temps d'entraînement **jusqu'à 10× plus rapide** que les méthodes GCN ou autoencoder |
| ✅ Robustesse | Capacité à ignorer les vues peu informatives via le mécanisme d'attention/inertie |
| ✅ Généralité | Supporte différents types de graphes multi-vues sans contraintes |
| ✅ Formulation unifiée | Apprentissage de représentation + clustering dans un même objectif |
| ✅ Reproductibilité | Code disponible en open-source et résultats détaillés sur 5 benchmarks |

## ⚠️ **Limites** de LMGEC

| Limite | Détail |
|--------|--------|
| ❌ Méthode linéaire | Ne capture pas les non-linéarités complexes, contrairement aux modèles deep |
| ❌ Pas de gestion des vues manquantes | Chaque vue est supposée complète et bien alignée |
| ❌ Risque de sur-lissage évité uniquement via un filtrage 1-hop | Ce choix reste rigide dans certains cas |
| ❌ Pas de mécanisme d'apprentissage end-to-end avec supervision éventuelle | Modèle strictement non supervisé |
| ❌ Pas de mécanisme explicite de fusion dynamique | Le poids est fixé après initialisation (pas appris pendant optimisation) |

## 🧠 Conclusion
LMGEC se positionne comme une **alternative simple, rapide et robuste** aux méthodes complexes basées sur GCN ou autoencodeurs. Il est particulièrement pertinent dans des contextes contraints en ressources ou nécessitant une interprétabilité forte.

Cependant, pour des cas très non-linéaires ou avec des données partiellement alignées, des méthodes plus expressives comme les modèles à attention ou graph contrastif peuvent être préférées.

## 🌐 Récapitulatif Synthétique

| Élément | Déscription |
|--------|-------------|
| **Fusion** | Tardive, linéaire, pondération adaptative |
| **Hypothèses sur les vues** | Mêmes nœuds, hétérogénéité supportée |
| **Formulation** | Linéaire, objectif joint reconstruction + clustering |
| **Données** | Multi-vues topologiques, attributaires ou mixtes |
| **Métriques** | NMI, ARI, ACC, F1 |

---

# Analyse de l’article "Efficient Graph Convolution for Joint Node Representation Learning and Clustering" (Fettal et al., WSDM 2022)

---

## 1. Problématique centrale
L’article traite le problème de l’**apprentissage conjoint des représentations de nœuds et du clustering dans les graphes attribués**. L’objectif est d’améliorer la qualité du clustering en intégrant directement l’objectif de regroupement dans la phase d’apprentissage des embeddings, tout en garantissant une **efficacité computationnelle**.

---

## 2. Type de fusion
- **Type :** Intermédiaire  
  La fusion des vues (structure + attributs) se fait via une fonction d’agrégation `agg(A, X) = TᵖX`, c’est-à-dire une **fusion intermédiaire**.
  
- **Nature :** Linéaire  
  La fusion est **linéaire** : l'agrégation est une multiplication de matrices entre une propagation modifiée $$T^p$$ et la matrice d'attributs $$X$$.

---

## 3. Pondération des vues
- Il n’y a **pas de pondération explicite des vues**. Le modèle utilise une **fusion fixe** de la structure (via le filtre $$T^p$$) et des attributs sans mécanisme d’attention ou de softmax.

---

## 4. Hypothèses sur les vues
- Les vues **partagent les mêmes nœuds** (les attributs sont associés aux mêmes sommets que ceux du graphe).
- Les vues peuvent avoir des **topologies différentes** (on peut appliquer différentes puissances de propagation $$p$$).
- Le modèle **ne traite pas explicitement les vues manquantes ou désalignées**.
- Il **réduit l’impact des vues bruitées** indirectement via le filtrage **low-pass** du signal (par normalisation spectrale du filtre).

---

## 5. Formalisme mathématique du modèle

### Fonction objectif (Eq. (1)) :
$$
\min_{\theta_1,\theta_2,G,F} \|\text{dec}_{\theta_2}(\text{enc}_{\theta_1}(T^p X)) - T^p X\|^2 + \alpha \|\text{enc}_{\theta_1}(T^p X) - GF\|^2
$$

- **1er terme :** reconstruction (type autoencodeur linéaire)
- **2e terme :** régularisation de clustering (k-means sur les embeddings)

### Contrainte :
$$
G \in \{0,1\}^{n \times k}, \quad G1_k = 1_n
$$
(matrice d’affectation de clusters : chaque nœud appartient à un et un seul cluster)

### Reformulation (Eq. (5), (6)) :
Hypothèse que $$W = W_1 = W_2^T$$, et que $$W^T W = I_k$$ (orthogonalité), on obtient :

$$
\min_{G,F,W} \|T^p X - GFW^T\|^2, \quad \text{s.c. } G \in \{0,1\}^{n \times k}, \, G1_k = 1_n, \, W^T W = I_k
$$

### Matrices utilisées :
- $ T $ : Matrice de propagation normalisée (modification de SGC)
- $ T^p X $ : signal propagé
- $ G $ : matrice d’assignation
- $ F $ : centroïdes
- $ W $ : projection orthogonale

### Optimisation :
- Alternance : mise à jour de $$G$$, $$F$$, et $$W$$ par descente coordonnée
- Méthodes :
  - $ F $ : least squares
  - $ W $ : Procrustes (SVD)
  - $ G $ : k-means

---

## 6. Données utilisées
- Données **réelles** : Cora, Citeseer, Pubmed, Wiki
- Les graphes ont les **mêmes nœuds** dans structure et attributs
- **Topologies différentes** : oui, entre structure (A) et attributs (X)
- Pas de **vues manquantes**, toutes les matrices sont complètes

---

## 7. Expériences
- **Métriques** : Accuracy, NMI, F1-score
- **Études d’ablation** :
  - Impact de la **matrice de propagation** (comparaison avec normalisations classiques)
  - Influence du **paramètre de propagation** $$p$$
- **Robustesse au bruit** : pas explicitement testée
- **Transductif** : le modèle apprend sur le graphe complet (pas inductif)

---

## 8. Avantages mis en avant
- **Formulation simple** et **analytique**
- **Complexité faible** $$O(p|E|d + tndk)$$
- **Performances comparables ou meilleures** que SOTA
- **Visualisation claire** via embeddings structurés ($$R^2$$ élevé)
- **Facilement implémentable** et rapide sur GPU

---

## 9. Limitations
- **Modèle transductif**, pas d’extension inductive
- **Paramètre $$p$$** sensible (propagation) → nécessite une sélection heuristique
- Pas de gestion explicite des **vues manquantes** ou des **données bruitées**

---

## 10. Perspectives d’amélioration
- Étendre à une **formulation inductive**
- Ajouter un **mécanisme adaptatif pour le poids $$\alpha$$** entre reconstruction et clustering
- Envisager des **versions co-clustering** pour documents/entités bipartites
- Ajouter **pondération ou attention** entre structure et attributs

---


# Analyse de l’article : Efficient Graph Convolution for Joint Node Representation Learning and Clustering (GCC)

---

## 1. Quelle est la problématique centrale à laquelle répond l’article ?

L’article traite **du clustering de graphes attribués**, où chaque nœud a des attributs. Le défi est de faire **l’apprentissage de représentations et le clustering conjointement**, plutôt que de manière séquentielle (tandem). La méthode vise à produire des **représentations adaptées à la structure de clusters**, tout en étant **efficace computationnellement**.

- 🧠 **Défi principal :** clustering efficace et conjoint avec l’apprentissage de représentations dans les graphes attribués, avec une complexité réduite.

![Q1](2-1.png)

---

## 2. Quel type de fusion est utilisé pour combiner les différentes vues ?

Même si GCC ne traite pas explicitement de données multivues, il applique une fusion **précoce linéaire** entre la topologie $A$ et les attributs $X$, via une opération d’agrégation :

- Type de fusion : **précoce**
- Nature de la fusion : **linéaire**
- Agrégation : $$ \text{agg}(A, X) = T^p X $$

![Q2](2-2.png)

---

## 3. Y a-t-il une pondération des vues ?

GCC ne traite qu’un seul graphe, donc **pas de pondération explicite entre vues**. Toutefois, le paramètre $p$ contrôle combien de "hops" sont pris en compte, ce qui agit comme une **pondération spatiale implicite**.

- ❌ Pas de softmax, attention ou pondération dynamique.
- ✅ Influence implicite via $p$.

![Q3](2-3.png)

---

## 4. Quelles sont les hypothèses faites sur les vues ?

Même dans un cadre monovue, GCC repose sur des hypothèses implicites :

- ✅ Les vues (topologie + attributs) **partagent les mêmes nœuds**.
- ❌ Ne supporte pas de **vues hétérogènes** ou **désalignées**.
- ❌ Ne gère pas les **vues manquantes**.
- ❌ Pas de mécanisme explicite contre le **bruit inter-vue**.

![Q4](2-4.png)

---

## 5. Quel est le formalisme mathématique du modèle ?

Le modèle repose sur une **optimisation conjointe** d’une perte de reconstruction et d’une régularisation de clustering :

### Fonction objectif principale :
$$
\min_{G, F, W} \| T^p X - T^p X W W^\top \|^2 + \alpha \| T^p X W - G F \|^2
$$

### Avec contraintes :
- $W^\top W = I_k$
- $G \in \{0, 1\}^{n \times k}$ et $G \mathbf{1}_k = \mathbf{1}_n$

On peut reformuler en :
$$
\min_{G, F, W} \| T^p X - G F W^\top \|^2
$$

### Méthode d’optimisation :
- Alternance :
  - $F \leftarrow (G^\top G)^{-1} G^\top T^p X W$
  - $W \leftarrow \text{SVD}( (T^p X)^\top G F )$
  - $G \leftarrow \arg\min_j \| (T^p X W)_i - f_j \|^2$

![Q5](2-5.png)

---

## 6. Quel type de données est utilisé pour l’évaluation ?

Les jeux de données sont **réels**, avec une topologie fixe et des attributs différents selon les graphes :

| Dataset   | # Nodes | # Edges | # Features | # Classes |
|-----------|---------|---------|------------|-----------|
| Citeseer  | 3327    | 4732    | 3703       | 6         |
| Cora      | 2708    | 5429    | 1433       | 7         |
| Pubmed    | 19717   | 44338   | 500        | 3         |
| Wiki      | 2405    | 17981   | 4973       | 17        |

- ✅ Même nœuds, topologies fixes
- ✅ Attributs textuels
- ✅ Jeux de données entièrement réels

---

## 7. Quelles expériences sont menées pour valider le modèle ?

- **Comparaison avec les baselines** (S2GC, GIC, DCN, etc.) sur ACC, NMI, F1
- **Ablation sur la normalisation** utilisée pour $T$
- **Visualisation des embeddings** via t-SNE + analyse R²
- **Temps d’exécution** comparé aux autres méthodes (AGE, S2GC…)
- **Sélection automatique du paramètre** $p$

> Le modèle est **transductif**, pas inductif.

---

## 8. Quels sont les avantages mis en avant par les auteurs ?

- ✅ **Simplicité** (modèle linéaire, formulation analytique)
- ✅ **Rapidité** (SVD, closed-form updates)
- ✅ **Optimisation déterministe** (descente garantie)
- ✅ **Représentations de qualité** pour clustering et visualisation
- ✅ **Scalabilité** prouvée sur des graphes de grande taille (ex : Pubmed)

---

## 9. Quelles sont les limitations ou points faibles du modèle ?

- ❌ Pas de version **inductive**
- ❌ Sensible à l’**initialisation**
- ❌ $p$ doit être **choisi heuristiquement**
- ❌ Ne supporte pas **vues multiples**, bruitées, ou **manquantes**

---

## 10. Quelles sont les perspectives d’amélioration discutées ?

- 🔄 Optimisation du **coefficient $\alpha$** entre reconstruction et clustering
- 🔮 Développement d’une version **inductive**
- 🧩 Extension vers le **co-clustering** (noeuds + attributs)
- 🌐 Adaptation plus fine du **paramètre $p$**

---

**Fin du rapport.**
